# Import libraries 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, accuracy_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.ensemble import RandomForestClassifier
import statistics as stats

In [3]:
import math

# Import data

In [4]:
data = pd.read_csv('../../raw_data/bitstampUSD.csv')

In [5]:
data

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4727772,1609372560,28801.47,28829.42,28785.64,28829.42,0.965221,27804.572129,28806.429798
4727773,1609372620,28829.42,28863.90,28829.42,28857.06,2.368831,68332.350629,28846.441863
4727774,1609372680,28850.49,28900.52,28850.49,28882.82,2.466590,71232.784464,28879.056266
4727775,1609372740,28910.54,28911.52,28867.60,28881.30,7.332773,211870.912660,28893.695831


# Nan's exploration

In [ ]:
sns.heatmap(data.isnull(), cbar=False)

To be continued !

# Preprocessing, input data, Model, Cross Val

In [6]:
def get_data():
    """method to get the training data (or a portion of it) from google cloud bucket"""
    # client = storage.Client()
    # data = pd.read_csv(f"gs://{BUCKET_NAME}/{BUCKET_TRAIN_DATA_PATH}")
    data = pd.read_csv('../../raw_data/bitstampUSD.csv')
    # data = pd.read_csv('gs://bitcoin-prediction-01/data/bitstampUSD.csv')
    data = data[2798176:4727776].copy()
    return data


In [7]:
def select_date(data, date_start, date_end):
    
    data['Timestamp'] = pd.to_datetime(data['Timestamp'], unit='s', origin='unix')
    data = data[['Open', 'Timestamp']].set_index("Timestamp").fillna(method='ffill')
    
    if date_start != None:
        if date_end != None:
            data = data[date_start:date_end].copy()
    else:
        data = data.copy()
        
    return data

In [ ]:
# select_data(data, "2020-8-1", "2020-10-2") 

In [8]:
def preprocessing_data(data, features_size, h):
        
    data_pp = data.copy()
    data_pp['diff_Open'] = data_pp['Open'].diff(h)
    data_pp['diff_Open'] = data_pp['diff_Open'].dropna()
    data_pp[f"t+{h}"] = data_pp['diff_Open'].shift(-h)
    
    for i in range(0, features_size):
        data_pp[f't-{i}'] = data_pp['Open'].shift(i)
    data_shifted = data_pp.dropna()
    
    return data_shifted

In [9]:
# preprocessing_data(data, 50, 2)

In [10]:
def features_target(data_shifted, h):
    
    X = data_shifted.drop(columns=['Open', 'diff_Open', f"t+{h}"])
    y = data_shifted[f"t+{h}"].copy()
    y[y > 0] = 1
    y[y <= 0] = 0
    
    data_size = data_shifted.shape[0]
    
    return X, y, data_size

In [11]:
def input_data(X, y, sample_size, train_fraction, features_size, data_size, train_size, test_size, h=1, w=0):    
 

    sample_X = X.iloc[data_size-(test_size * w + sample_size) : data_size - (test_size * w)]
    sample_y = y.iloc[data_size-(test_size * w + sample_size) : data_size - (test_size * w)]
    
    X_train = sample_X.iloc[0:train_size]
    y_train = sample_y.iloc[0:train_size]
    X_test = sample_X.iloc[(train_size+h-1):(sample_size)]
    y_test = sample_y.iloc[(train_size+h-1):(sample_size)]
    
    return X_train, X_test, y_train, y_test

In [ ]:
def predict_score(model_init, X_train, X_test, y_train, y_test):
    model = model_init
    model = model.fit(X_train, y_train)
    results = model.predict(X_test)
    score = model.score(X_test, y_test) 
    
    return score

In [ ]:
def cross_val(model_init=RidgeClassifier(), sample_size=1000, train_fraction=0.7, features_size=60, h=1, date_start=None, date_end=None):
    
    data = get_data()
#     data = select_date(data, date_start, date_end)
#     data_shifted = preprocessing_data(data, features_size, h)
#     X, y, data_size = features_target(data_shifted, h)
#     train_size = int(train_fraction*sample_size)
#     test_size = sample_size - train_size
    
    
#     r = math.floor((data_size-train_size)/test_size)
#     intervals = range(0, r)
#     reversed_intervals = reversed(intervals)
#     results = []
    
#     for i in reversed_intervals:
#         X_train, X_test, y_train, y_test = input_data(X, y, sample_size, train_fraction, features_size, data_size, train_size, test_size, h, w=i)
#         score = predict_score(model_init, X_train, X_test, y_train, y_test)
#         results.append(score)
    
    return data
#     return dict({'mean_score':round(stats.mean(results),2), 'std':round(stats.stdev(results),2) , 'score_min':round(min(results),2), 'score_max':round(max(results),2)})
    

In [ ]:
cross_val(model_init=RidgeClassifier(), sample_size=1000, train_fraction=0.7, features_size=60, h=1, date_start=None, date_end=None)

In [ ]:
60*24*30*44

In [ ]:
(1900800-700)/300

In [ ]:
((60*24*7)-700)/300

In [ ]:
cross_val(model_init=RidgeClassifier(), sample_size=1000, train_fraction=0.7, features_size=60, h=1, date_start=None, date_end=None)

In [ ]:
cross_val(model_init=RidgeClassifier(), sample_size=1000, train_fraction=0.7, features_size=60, h=1, date_start=None, date_end=None)

In [ ]:
cross_val(model_init=RidgeClassifier(), sample_size=1000, train_fraction=0.7, features_size=60, h=30, date_start=None, date_end=None)

In [ ]:
cross_val(model_init=RidgeClassifier(), sample_size=1440, train_fraction=0.7, features_size=60, h=10, date_start=None, date_end=None)

In [ ]:
cross_val(model_init=RidgeClassifier(), sample_size=1440, train_fraction=0.7, features_size=60, h=5, date_start=None, date_end=None)

In [ ]:
cross_val(model_init=RidgeClassifier(), sample_size=1440, train_fraction=0.7, features_size=60, h=2, date_start=None, date_end=None)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, Dense

In [13]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

In [ ]:
# model = Sequential()
# model.add(layers.SimpleRNN(units=1, activation='tanh')) # we'll look at units number later
# model.add(layers.Dense(1, activation="relu"))

# # The compilation
# model.compile(loss='mse', 
#               optimizer='rmsprop')

# # The fit
# model.fit(X, y,
#          batch_size=16,
#          epochs=10, verbose=0)

# # The prediction
# model.predict(X)

In [ ]:
def test_Xy(sample_size=1000, train_fraction=0.7, features_size=60, h=1, w=0, RNN=False, date_start=None, date_end=None):
    
    data = get_data()
    data = select_date(data, date_start, date_end)
    data_shifted = preprocessing_data(data, features_size, h)
    X, y, data_size = features_target(data_shifted, h)
    train_size = int(train_fraction*sample_size)
    test_size = sample_size - train_size
    X_train, X_test, y_train, y_test = input_data(X, y, sample_size, train_fraction, features_size, data_size, train_size, test_size, h, w)
    
    if RNN == True:
        X_train, y_train = np.array(X_train), np.array(y_train)
        X_test, y_test = np.array(X_test), np.array(y_test)
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = test_Xy(sample_size=1440, train_fraction=0.7, features_size=60, h=2, w=0, RNN=True, date_start=None, date_end=None)

In [ ]:
# model = Sequential()
# model.add(LSTM(10, activation='tanh'))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=16)

In [ ]:
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))
# Compiling the RNN
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
# model = Sequential()
# model.add(Dense(100, activation='tanh'))
# model.add(Dense(50, activation='tanh'))
# model.add(Dense(10, activation='tanh'))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=16)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
scores

In [ ]:
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 40, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 30, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 15))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))
# Compiling the RNN
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs = 50, batch_size = 32)

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
scores[1]

In [ ]:
# es = EarlyStopping(patience=20, restore_best_weights=True)

# model.fit(X_train, y_train, 
#           batch_size=16, 
#           epochs=1000, 
#           validation_split=0.3,
#           callbacks=[es])

In [14]:
def predict_score_deep(model_deep, X_train, X_test, y_train, y_test):
    
    model = Sequential()
    es = EarlyStopping(patience=2, restore_best_weights=True)
    
    if model_deep == LSTM:
        model.add(LSTM(units = 30, activation='tanh', return_sequences = True, input_shape = (X_train.shape[1], 1)))
        model.add(Dropout(0.2))
#         model.add(LSTM(units = 40, return_sequences = True))
#         model.add(Dropout(0.2))
        model.add(LSTM(units = 10,  activation='tanh', return_sequences = True))
        model.add(Dropout(0.2))
        # Adding the output layer
        model.add(Dense(units = 5, activation='relu'))
        model.add(Dense(units = 1, activation='sigmoid'))
        # Compiling the RNN
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        # Fitting the RNN to the Training set
        model.fit(X_train, y_train, validation_split=0.2, epochs = 100, batch_size = 32, callbacks=[es])

        score = model.evaluate(X_test, y_test, verbose=0)
    
    if model_deep == GRU:
        pass
        
    return score[1] #attention score[0] loss à return also

In [ ]:
# def deep_test_b(model_deep, sample_size=1000, train_fraction=0.7, features_size=60, h=1, RNN=True, date_start=None, date_end=None):
    
#     data = get_data()
#     data = select_date(data, date_start, date_end)
#     data_shifted = preprocessing_data(data, features_size, h)
#     X, y, data_size = features_target(data_shifted, h)
#     train_size = int(train_fraction*sample_size)
#     test_size = sample_size - train_size
    
#     if RNN == True:
#             X_train, y_train = np.array(X_train), np.array(y_train)
#             X_test, y_test = np.array(X_test), np.array(y_test)
#             X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
#             X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    

#     X_train, X_test, y_train, y_test = input_data(X, y, sample_size, train_fraction, features_size, data_size, train_size, test_size, h, w=0)

    
#     return X_train, X_test, y_train, y_test

In [ ]:
1440*7

In [ ]:
# X_train, X_test, y_train, y_test = deep_test_b(LSTM, sample_size=100, train_fraction=0.7, features_size=50, h=5, RNN=True, date_start="2020", date_end="2020")

In [ ]:
X_train, X_test, y_train, y_test = deep_test_b(LSTM, sample_size=10080, train_fraction=0.7, features_size=21600, h=1440, RNN=True, date_start="2020", date_end="2020")

In [ ]:
X_train.shape

In [ ]:
model = Sequential()
model.add(LSTM(units = 30, activation='tanh', return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units = 10,  activation='tanh', return_sequences = True))
model.add(Dropout(0.2))
model.add(Dense(units = 5, activation='relu'))
model.add(Dense(units = 1, activation='sigmoid'))

In [ ]:
model.summary()

In [15]:
def deep_test(model_deep, sample_size=1000, train_fraction=0.7, features_size=60, h=1, RNN=True, date_start=None, date_end=None):
    
    data = get_data()
    data = select_date(data, date_start, date_end)
    data_shifted = preprocessing_data(data, features_size, h)
    X, y, data_size = features_target(data_shifted, h)
    train_size = int(train_fraction*sample_size)
    test_size = sample_size - train_size
    
    r = math.floor((data_size-train_size)/test_size)
    intervals = range(0, r)
    reversed_intervals = reversed(intervals)
    results = []
    
    for i in reversed_intervals:
        X_train, X_test, y_train, y_test = input_data(X, y, sample_size, train_fraction, features_size, data_size, train_size, test_size, h, w=i)
        
        if RNN == True:
            X_train, y_train = np.array(X_train), np.array(y_train)
            X_test, y_test = np.array(X_test), np.array(y_test)
            X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
            X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
            
        score = predict_score_deep(model_deep, X_train, X_test, y_train, y_test)
        results.append(score)
    
    return dict({'mean_score':round(stats.mean(results),2),
                 'std':round(stats.stdev(results),2),
                 'score_min':round(min(results),2),
                 'score_max':round(max(results),2),
                 'n_fold':r})
    
    
    

In [ ]:
1440*30

In [17]:
deep_test(LSTM, sample_size=43200, train_fraction=0.7, features_size=21600, h=1440, RNN=True, date_start="2020", date_end="2020")

MemoryError: Unable to allocate 5.09 GiB for an array with shape (1301, 525600) and data type float64

In [ ]:
deep_test(LSTM, sample_size=1440, train_fraction=0.7, features_size=60, h=10, w=0, RNN=True, date_start="2020-01-01", date_end="2020-02-01")

In [ ]:
notebook résultats
d-tale
recuperer données mlflow, filtre
deep
tester modeles

In [ ]:
# create model
# modeldeep_2 = KerasClassifier(build_fn=create_model)

In [ ]:
# cross_val(model_init=modeldeep, sample_size=1440, train_fraction=0.7, features_size=60, h=2, date_start=None, date_end=None)